# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [10]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [11]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [12]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




*** 

## Model Construction

### Table 1 Structure

**Primary Key** - (session_id, item_in_session) Partition by session_id, cluster by item_in_session

**Structure**
* session_id *bigint*
* item_in_session *int*
* artist_name *text*
* song_title *text*
* song_length *decimal*

### Table 2 Structure

**Primary Key** - ((user_id, session_id), item_in_session) Partition by user_id and session_id, cluster by item_in_session
Note: Since a user may have multiple song plays user_id alone would form a poor partition key given it would generate unbalanced data distribution.

**Structure**
* user_id *bigint*
* session_id *bigint*
* item_in_session *int*
* artist_name *text*
* song_title *text*
* first_name *text*
* last_name *text*

### Table 3 Structure

**Primary Key** - (song_title, user_id) Partition by song_title, cluster by user_id

**Structure**
* song_title *text*
* user_id *bigint*
* item_in_session *int*
* first_name *text*
* last_name *text*

In [45]:
from typing import List, Tuple

query1_table: str = "CREATE TABLE IF NOT EXISTS music_sessions"
query1_table += " (session_id bigint, item_in_session int, artist_name text, song_title text, song_length decimal, PRIMARY KEY (session_id, item_in_session))"

query2_table: str = "CREATE TABLE IF NOT EXISTS music_users"
query2_table += " (user_id bigint, session_id bigint, item_in_session int, artist_name text, song_title text, first_name text, last_name text,PRIMARY KEY ((user_id, session_id), item_in_session))"
    
query3_table: str = "CREATE TABLE IF NOT EXISTS music_history"
query3_table += " (song_title text, user_id bigint, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"

queries: List[str] = [query1_table, query2_table, query3_table]
    
try:
    for query in queries:
        session.execute(query)
except Exception as e:
    print(e)

### Data Insertion

In [46]:
file = 'event_datafile_new.csv'

insertion_query1: str = "INSERT INTO music_sessions (session_id, item_in_session, artist_name, song_title, song_length) values (%s, %s, %s, %s, %s)"
insertion_query2: str = "INSERT INTO music_users (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) values (%s, %s, %s, %s, %s, %s, %s)"
insertion_query3: str = "INSERT INTO music_history (song_title, user_id, first_name, last_name) values (%s, %s, %s, %s)"
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        user_id: int = int(line[10])
        session_id: int = int(line[8])
        item_in_session: int = int(line[3])
        artist_name: str = line[0]
        song_title: str = line[9]
        song_length: str = float(line[5])
        first_name: str = line[1]
        last_name: str = line[4]
        
        insertion_elements_query1: Tuple[int, int, str, str, float] = (session_id, item_in_session, artist_name, song_title, song_length)
        insertion_elements_query2: Tuple[int, int, int, str, str, str, str] = (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)
        insertion_elements_query3: Tuple[str, int, str, str] = (song_title, user_id, first_name, last_name)

        try:
            session.execute(insertion_query1, insertion_elements_query1)
            session.execute(insertion_query2, insertion_elements_query2)
            session.execute(insertion_query3, insertion_elements_query3)
        except Exception as e:
            print(e)

### Query 1

#### Retrieve the song play information regarding session 338 for item in session 4:

Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [47]:
query_1: str = "SELECT artist_name, song_title, song_length FROM music_sessions WHERE session_id = 338 and item_in_session = 4"
    
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### Query 2

#### Retrieve the songs listened by user 10 within session 182:

"Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182"

In [48]:
query_2: str = "SELECT artist_name, song_title, first_name, last_name FROM music_users WHERE user_id = 10 and session_id = 182"
    
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3

#### Retrieve the list of users who listened to the song 'All Hands Against His Own'

"Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'"

In [50]:
query_3: str = "SELECT first_name, last_name FROM music_history WHERE song_title = 'All Hands Against His Own'"
    
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [51]:
query1_drop_table: str = "DROP TABLE IF EXISTS music_sessions"
query2_drop_table: str = "DROP TABLE IF EXISTS music_users"
query3_drop_table: str = "DROP TABLE IF EXISTS music_history"

queries: List[str] = [query1_drop_table, query2_drop_table, query3_drop_table]
    
try:
    for query in queries:
        session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [52]:
session.shutdown()
cluster.shutdown()